# Ablation

In [1]:
import os
import random
import numpy as np
import torch as tr
import pandas as pd
import shutil

import torch.nn as nn
from sincfold.dataset import SeqDataset, pad_batch
from torch.utils.data import DataLoader
from sincfold.model import sincfold

from sincfold.ablation.ablation_1ResNet2d import sincfold_1ResNet2d
from sincfold.ablation.ablation_no_ResNet1d_FF import sincfold_no_ResNet1d_FF
from sincfold.ablation.ablation_no_ResNet2d import sincfold_no_ResNet2d
from sincfold.ablation.ablation_C1D_C2D import sincfold_C1D_C2D
from sincfold.ablation.ablation_no_ResNet1d import sincfold_no_ResNet1d
 

In [2]:
# Reproducibility
tr.manual_seed(42)
random.seed(42)
np.random.seed(42)

In [3]:
# Train Params
train_file ='../data/ArchiveII_sample.csv'
config={}
valid_file=None
nworkers=2
verbose=True

## Sincfold

In [4]:
out_path='./pruebas/original'


if verbose:
    print("Working on", out_path)

if "cache_path" not in config:
    config["cache_path"] = "cache/"

if not os.path.isdir(out_path):
    os.makedirs(out_path)
else:
    raise ValueError(f"Output path {out_path} already exists")

if valid_file is not None:
    train_file = train_file
    valid_file = valid_file
else:
    data = pd.read_csv(train_file)
    valid_split = config["valid_split"] if "valid_split" in config else 0.1
    train_file = os.path.join(out_path, "train.csv")
    valid_file = os.path.join(out_path, "valid.csv")

    val_data = data.sample(frac = valid_split)
    val_data.to_csv(valid_file, index=False)
    data.drop(val_data.index).to_csv(train_file, index=False)
    
batch_size = config["batch_size"] if "batch_size" in config else 4
train_loader = DataLoader(
    SeqDataset(train_file, training=True, **config),
    batch_size=batch_size, 
    shuffle=True,
    num_workers=nworkers,
    collate_fn=pad_batch
)
valid_loader = DataLoader(
    SeqDataset(valid_file, **config),
    batch_size=batch_size,
    shuffle=False,
    num_workers=nworkers,
    collate_fn=pad_batch,
)

net = sincfold(train_len=len(train_loader), **config)

best_f1, patience_counter = -1, 0
patience = config["patience"] if "patience" in config else 30
if verbose:
    print("Start training...")
max_epochs = config["max_epochs"] if "max_epochs" in config else 1000
logfile = os.path.join(out_path, "train_log.csv") 
    
for epoch in range(6): ## 6 epochs
    train_metrics = net.fit(train_loader)

    val_metrics = net.test(valid_loader)

    if val_metrics["f1"] > best_f1:
        best_f1 = val_metrics["f1"]
        tr.save(net.state_dict(), os.path.join(out_path, "weights.pmt"))
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter > patience:
            break
    
    if not os.path.exists(logfile):
        with open(logfile, "w") as f: 
            msg = ','.join(['epoch']+[f"train_{k}" for k in sorted(train_metrics.keys())]+[f"valid_{k}" for k in sorted(val_metrics.keys())]) + "\n"
            f.write(msg)
            f.flush()
            if verbose:
                print(msg)

    with open(logfile, "a") as f: 
        msg = ','.join([str(epoch)]+[f'{train_metrics[k]:.4f}' for k in sorted(train_metrics.keys())]+[f'{val_metrics[k]:.4f}' for k in sorted(val_metrics.keys())]) + "\n"
        f.write(msg)
        f.flush()    
        if verbose:
            print(msg)
        
# remove temporal files           
shutil.rmtree(config["cache_path"], ignore_errors=True)

tmp_file = os.path.join(out_path, "train.csv")
if os.path.exists(tmp_file):
    os.remove(tmp_file)
tmp_file = os.path.join(out_path, "valid.csv")
if os.path.exists(tmp_file):
    os.remove(tmp_file)

Working on ./pruebas/original
No weights provided, using random initialization
Start training...


100%|██████████| 1/1 [00:01<00:00,  1.32s/it]


epoch,train_f1,train_loss,valid_f1,valid_f1_post,valid_loss

0,0.0016,0.5476,0.0000,0.0000,0.1929



100%|██████████| 1/1 [00:01<00:00,  1.46s/it]


1,0.0000,0.2032,0.0000,0.0000,0.2465



...

3,0.0000,0.1556,0.0000,0.0000,0.2045



100%|██████████| 1/1 [00:01<00:00,  1.67s/it]


4,0.0000,0.1079,0.0000,0.0000,0.1705



100%|██████████| 1/1 [00:01<00:00,  1.57s/it]

5,0.0000,0.0802,0.0000,0.0000,0.1437



## ablation: 1 ResNet2d

In [11]:

train_file ='../data/ArchiveII_sample.csv'

valid_file=None
out_path='./pruebas/1_ResNet2d/'


if verbose:
    print("Working on", out_path)

if "cache_path" not in config:
    config["cache_path"] = "cache/"

if not os.path.isdir(out_path):
    os.makedirs(out_path)
else:
    raise ValueError(f"Output path {out_path} already exists")

if valid_file is not None:
    train_file = train_file
    valid_file = valid_file
else:
    data = pd.read_csv(train_file)
    valid_split = config["valid_split"] if "valid_split" in config else 0.1
    train_file = os.path.join(out_path, "train.csv")
    valid_file = os.path.join(out_path, "valid.csv")

    val_data = data.sample(frac = valid_split)
    val_data.to_csv(valid_file, index=False)
    data.drop(val_data.index).to_csv(train_file, index=False)
    
batch_size = config["batch_size"] if "batch_size" in config else 4
train_loader = DataLoader(
    SeqDataset(train_file, training=True, **config),
    batch_size=batch_size, 
    shuffle=True,
    num_workers=nworkers,
    collate_fn=pad_batch
)
valid_loader = DataLoader(
    SeqDataset(valid_file, **config),
    batch_size=batch_size,
    shuffle=False,
    num_workers=nworkers,
    collate_fn=pad_batch,
)

net = sincfold_1ResNet2d(train_len=len(train_loader), **config)

best_f1, patience_counter = -1, 0
patience = config["patience"] if "patience" in config else 30
if verbose:
    print("Start training...")
max_epochs = config["max_epochs"] if "max_epochs" in config else 1000
logfile = os.path.join(out_path, "train_log.csv") 
    
for epoch in range(6): ## 6 epochs
    train_metrics = net.fit(train_loader)

    val_metrics = net.test(valid_loader)

    if val_metrics["f1"] > best_f1:
        best_f1 = val_metrics["f1"]
        tr.save(net.state_dict(), os.path.join(out_path, "weights.pmt"))
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter > patience:
            break
    
    if not os.path.exists(logfile):
        with open(logfile, "w") as f: 
            msg = ','.join(['epoch']+[f"train_{k}" for k in sorted(train_metrics.keys())]+[f"valid_{k}" for k in sorted(val_metrics.keys())]) + "\n"
            f.write(msg)
            f.flush()
            if verbose:
                print(msg)

    with open(logfile, "a") as f: 
        msg = ','.join([str(epoch)]+[f'{train_metrics[k]:.4f}' for k in sorted(train_metrics.keys())]+[f'{val_metrics[k]:.4f}' for k in sorted(val_metrics.keys())]) + "\n"
        f.write(msg)
        f.flush()    
        if verbose:
            print(msg)
        
# remove temporal files           
shutil.rmtree(config["cache_path"], ignore_errors=True)

tmp_file = os.path.join(out_path, "train.csv")
if os.path.exists(tmp_file):
    os.remove(tmp_file)
tmp_file = os.path.join(out_path, "valid.csv")
if os.path.exists(tmp_file):
    os.remove(tmp_file)

Working on ./pruebas/1_ResNet2d/
No weights provided, using random initialization
Start training...


100%|██████████| 1/1 [00:01<00:00,  1.41s/it]


epoch,train_f1,train_loss,valid_f1,valid_f1_post,valid_loss

0,0.0002,0.1727,0.0000,0.0000,0.2271



100%|██████████| 1/1 [00:01<00:00,  1.26s/it]


1,0.0000,0.1579,0.0000,0.0000,0.2319



...

3,0.0000,0.0961,0.0000,0.0000,0.1708



100%|██████████| 1/1 [00:01<00:00,  1.44s/it]


4,0.0198,0.0655,0.0556,0.0556,0.1565



100%|██████████| 1/1 [00:01<00:00,  1.19s/it]

5,0.1438,0.0544,0.2821,0.2105,0.1578



## sincfold_no_ResNet1d_FF

In [5]:
train_file ='../data/ArchiveII_sample.csv'

valid_file=None
out_path='./pruebas/no_ResNet1d_FF/'


if verbose:
    print("Working on", out_path)

if "cache_path" not in config:
    config["cache_path"] = "cache/"

if not os.path.isdir(out_path):
    os.makedirs(out_path)
else:
    raise ValueError(f"Output path {out_path} already exists")

if valid_file is not None:
    train_file = train_file
    valid_file = valid_file
else:
    data = pd.read_csv(train_file)
    valid_split = config["valid_split"] if "valid_split" in config else 0.1
    train_file = os.path.join(out_path, "train.csv")
    valid_file = os.path.join(out_path, "valid.csv")

    val_data = data.sample(frac = valid_split)
    val_data.to_csv(valid_file, index=False)
    data.drop(val_data.index).to_csv(train_file, index=False)
    
batch_size = config["batch_size"] if "batch_size" in config else 4
train_loader = DataLoader(
    SeqDataset(train_file, training=True, **config),
    batch_size=batch_size, 
    shuffle=True,
    num_workers=nworkers,
    collate_fn=pad_batch
)
valid_loader = DataLoader(
    SeqDataset(valid_file, **config),
    batch_size=batch_size,
    shuffle=False,
    num_workers=nworkers,
    collate_fn=pad_batch,
)

net = sincfold_no_ResNet1d_FF(train_len=len(train_loader), **config)

best_f1, patience_counter = -1, 0
patience = config["patience"] if "patience" in config else 30
if verbose:
    print("Start training...")
max_epochs = config["max_epochs"] if "max_epochs" in config else 1000
logfile = os.path.join(out_path, "train_log.csv") 
    
for epoch in range(6): ## 6 epochs
    train_metrics = net.fit(train_loader)

    val_metrics = net.test(valid_loader)

    if val_metrics["f1"] > best_f1:
        best_f1 = val_metrics["f1"]
        tr.save(net.state_dict(), os.path.join(out_path, "weights.pmt"))
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter > patience:
            break
    
    if not os.path.exists(logfile):
        with open(logfile, "w") as f: 
            msg = ','.join(['epoch']+[f"train_{k}" for k in sorted(train_metrics.keys())]+[f"valid_{k}" for k in sorted(val_metrics.keys())]) + "\n"
            f.write(msg)
            f.flush()
            if verbose:
                print(msg)

    with open(logfile, "a") as f: 
        msg = ','.join([str(epoch)]+[f'{train_metrics[k]:.4f}' for k in sorted(train_metrics.keys())]+[f'{val_metrics[k]:.4f}' for k in sorted(val_metrics.keys())]) + "\n"
        f.write(msg)
        f.flush()    
        if verbose:
            print(msg)
        
# remove temporal files           
shutil.rmtree(config["cache_path"], ignore_errors=True)

tmp_file = os.path.join(out_path, "train.csv")
if os.path.exists(tmp_file):
    os.remove(tmp_file)
tmp_file = os.path.join(out_path, "valid.csv")
if os.path.exists(tmp_file):
    os.remove(tmp_file)

Working on ./pruebas/no_ResNet1d_FF/
No weights provided, using random initialization
Start training...


  0%|          | 0/3 [00:02<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x356 and 4x32)


## sincfold_no_ResNet2d

In [13]:

train_file ='../data/ArchiveII_sample.csv'

valid_file=None
out_path='./pruebas/no_ResNet2d/'


if verbose:
    print("Working on", out_path)

if "cache_path" not in config:
    config["cache_path"] = "cache/"

if not os.path.isdir(out_path):
    os.makedirs(out_path)
else:
    raise ValueError(f"Output path {out_path} already exists")

if valid_file is not None:
    train_file = train_file
    valid_file = valid_file
else:
    data = pd.read_csv(train_file)
    valid_split = config["valid_split"] if "valid_split" in config else 0.1
    train_file = os.path.join(out_path, "train.csv")
    valid_file = os.path.join(out_path, "valid.csv")

    val_data = data.sample(frac = valid_split)
    val_data.to_csv(valid_file, index=False)
    data.drop(val_data.index).to_csv(train_file, index=False)
    
batch_size = config["batch_size"] if "batch_size" in config else 4
train_loader = DataLoader(
    SeqDataset(train_file, training=True, **config),
    batch_size=batch_size, 
    shuffle=True,
    num_workers=nworkers,
    collate_fn=pad_batch
)
valid_loader = DataLoader(
    SeqDataset(valid_file, **config),
    batch_size=batch_size,
    shuffle=False,
    num_workers=nworkers,
    collate_fn=pad_batch,
)

net = sincfold_no_ResNet2d(train_len=len(train_loader), **config)

best_f1, patience_counter = -1, 0
patience = config["patience"] if "patience" in config else 30
if verbose:
    print("Start training...")
max_epochs = config["max_epochs"] if "max_epochs" in config else 1000
logfile = os.path.join(out_path, "train_log.csv") 
    
for epoch in range(6): ## 6 epochs
    train_metrics = net.fit(train_loader)

    val_metrics = net.test(valid_loader)

    if val_metrics["f1"] > best_f1:
        best_f1 = val_metrics["f1"]
        tr.save(net.state_dict(), os.path.join(out_path, "weights.pmt"))
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter > patience:
            break
    
    if not os.path.exists(logfile):
        with open(logfile, "w") as f: 
            msg = ','.join(['epoch']+[f"train_{k}" for k in sorted(train_metrics.keys())]+[f"valid_{k}" for k in sorted(val_metrics.keys())]) + "\n"
            f.write(msg)
            f.flush()
            if verbose:
                print(msg)

    with open(logfile, "a") as f: 
        msg = ','.join([str(epoch)]+[f'{train_metrics[k]:.4f}' for k in sorted(train_metrics.keys())]+[f'{val_metrics[k]:.4f}' for k in sorted(val_metrics.keys())]) + "\n"
        f.write(msg)
        f.flush()    
        if verbose:
            print(msg)
        
# remove temporal files           
shutil.rmtree(config["cache_path"], ignore_errors=True)

tmp_file = os.path.join(out_path, "train.csv")
if os.path.exists(tmp_file):
    os.remove(tmp_file)
tmp_file = os.path.join(out_path, "valid.csv")
if os.path.exists(tmp_file):
    os.remove(tmp_file)

Working on ./pruebas/no_ResNet2d/
No weights provided, using random initialization
Start training...


100%|██████████| 1/1 [00:09<00:00,  9.74s/it]


epoch,train_f1,train_loss,valid_f1,valid_f1_post,valid_loss

0,0.0022,0.5878,0.0000,0.0000,0.0897



100%|██████████| 1/1 [00:08<00:00,  8.97s/it]


1,0.0000,0.2281,0.0000,0.0000,0.1161



...

3,0.0000,0.2432,0.0000,0.0000,0.0950



100%|██████████| 1/1 [00:13<00:00, 13.23s/it]


4,0.0000,0.2407,0.0000,0.0000,0.0781



100%|██████████| 1/1 [00:13<00:00, 13.61s/it]

5,0.0000,0.1362,0.0000,0.0000,0.0681




## sincfold_C1D_C2D

In [8]:

train_file ='../data/ArchiveII_sample.csv'

valid_file=None
out_path='./pruebas/C1D_C2D/'


if verbose:
    print("Working on", out_path)

if "cache_path" not in config:
    config["cache_path"] = "cache/"

if not os.path.isdir(out_path):
    os.makedirs(out_path)
else:
    raise ValueError(f"Output path {out_path} already exists")

if valid_file is not None:
    train_file = train_file
    valid_file = valid_file
else:
    data = pd.read_csv(train_file)
    valid_split = config["valid_split"] if "valid_split" in config else 0.1
    train_file = os.path.join(out_path, "train.csv")
    valid_file = os.path.join(out_path, "valid.csv")

    val_data = data.sample(frac = valid_split)
    val_data.to_csv(valid_file, index=False)
    data.drop(val_data.index).to_csv(train_file, index=False)
    
batch_size = config["batch_size"] if "batch_size" in config else 4
train_loader = DataLoader(
    SeqDataset(train_file, training=True, **config),
    batch_size=batch_size, 
    shuffle=True,
    num_workers=nworkers,
    collate_fn=pad_batch
)
valid_loader = DataLoader(
    SeqDataset(valid_file, **config),
    batch_size=batch_size,
    shuffle=False,
    num_workers=nworkers,
    collate_fn=pad_batch,
)

net = sincfold_C1D_C2D(train_len=len(train_loader), **config)

best_f1, patience_counter = -1, 0
patience = config["patience"] if "patience" in config else 30
if verbose:
    print("Start training...")
max_epochs = config["max_epochs"] if "max_epochs" in config else 1000
logfile = os.path.join(out_path, "train_log.csv") 
    
for epoch in range(6): ## 6 epochs
    train_metrics = net.fit(train_loader)

    val_metrics = net.test(valid_loader)

    if val_metrics["f1"] > best_f1:
        best_f1 = val_metrics["f1"]
        tr.save(net.state_dict(), os.path.join(out_path, "weights.pmt"))
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter > patience:
            break
    
    if not os.path.exists(logfile):
        with open(logfile, "w") as f: 
            msg = ','.join(['epoch']+[f"train_{k}" for k in sorted(train_metrics.keys())]+[f"valid_{k}" for k in sorted(val_metrics.keys())]) + "\n"
            f.write(msg)
            f.flush()
            if verbose:
                print(msg)

    with open(logfile, "a") as f: 
        msg = ','.join([str(epoch)]+[f'{train_metrics[k]:.4f}' for k in sorted(train_metrics.keys())]+[f'{val_metrics[k]:.4f}' for k in sorted(val_metrics.keys())]) + "\n"
        f.write(msg)
        f.flush()    
        if verbose:
            print(msg)
        
# remove temporal files           
shutil.rmtree(config["cache_path"], ignore_errors=True)

tmp_file = os.path.join(out_path, "train.csv")
if os.path.exists(tmp_file):
    os.remove(tmp_file)
tmp_file = os.path.join(out_path, "valid.csv")
if os.path.exists(tmp_file):
    os.remove(tmp_file)

Working on ./pruebas/C1D_C2D/
No weights provided, using random initialization
Start training...


100%|██████████| 1/1 [00:01<00:00,  1.37s/it]


epoch,train_f1,train_loss,valid_f1,valid_f1_post,valid_loss

0,0.0148,0.8448,0.0000,0.0000,0.4140



100%|██████████| 1/1 [00:00<00:00,  1.76it/s]


1,0.0014,0.3265,0.0000,0.0000,0.1648



...

3,0.0000,0.1203,0.0000,0.0000,0.0839



100%|██████████| 1/1 [00:00<00:00,  1.59it/s]


4,0.0000,0.1768,0.0000,0.0000,0.0819



100%|██████████| 1/1 [00:00<00:00,  1.54it/s]

5,0.0000,0.1651,0.0000,0.0000,0.0826




## sincfold_no_ResNet1d

In [7]:

train_file ='../data/ArchiveII_sample.csv'

valid_file=None
out_path='./pruebas/no_ResNet1d/'


if verbose:
    print("Working on", out_path)

if "cache_path" not in config:
    config["cache_path"] = "cache/"

if not os.path.isdir(out_path):
    os.makedirs(out_path)
else:
    raise ValueError(f"Output path {out_path} already exists")

if valid_file is not None:
    train_file = train_file
    valid_file = valid_file
else:
    data = pd.read_csv(train_file)
    valid_split = config["valid_split"] if "valid_split" in config else 0.1
    train_file = os.path.join(out_path, "train.csv")
    valid_file = os.path.join(out_path, "valid.csv")

    val_data = data.sample(frac = valid_split)
    val_data.to_csv(valid_file, index=False)
    data.drop(val_data.index).to_csv(train_file, index=False)
    
batch_size = config["batch_size"] if "batch_size" in config else 4
train_loader = DataLoader(
    SeqDataset(train_file, training=True, **config),
    batch_size=batch_size, 
    shuffle=True,
    num_workers=nworkers,
    collate_fn=pad_batch
)
valid_loader = DataLoader(
    SeqDataset(valid_file, **config),
    batch_size=batch_size,
    shuffle=False,
    num_workers=nworkers,
    collate_fn=pad_batch,
)

net = sincfold_no_ResNet1d(train_len=len(train_loader), **config)

best_f1, patience_counter = -1, 0
patience = config["patience"] if "patience" in config else 30
if verbose:
    print("Start training...")
max_epochs = config["max_epochs"] if "max_epochs" in config else 1000
logfile = os.path.join(out_path, "train_log.csv") 
    
for epoch in range(6): ## 6 epochs
    train_metrics = net.fit(train_loader)

    val_metrics = net.test(valid_loader)

    if val_metrics["f1"] > best_f1:
        best_f1 = val_metrics["f1"]
        tr.save(net.state_dict(), os.path.join(out_path, "weights.pmt"))
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter > patience:
            break
    
    if not os.path.exists(logfile):
        with open(logfile, "w") as f: 
            msg = ','.join(['epoch']+[f"train_{k}" for k in sorted(train_metrics.keys())]+[f"valid_{k}" for k in sorted(val_metrics.keys())]) + "\n"
            f.write(msg)
            f.flush()
            if verbose:
                print(msg)

    with open(logfile, "a") as f: 
        msg = ','.join([str(epoch)]+[f'{train_metrics[k]:.4f}' for k in sorted(train_metrics.keys())]+[f'{val_metrics[k]:.4f}' for k in sorted(val_metrics.keys())]) + "\n"
        f.write(msg)
        f.flush()    
        if verbose:
            print(msg)
        
# remove temporal files           
shutil.rmtree(config["cache_path"], ignore_errors=True)

tmp_file = os.path.join(out_path, "train.csv")
if os.path.exists(tmp_file):
    os.remove(tmp_file)
tmp_file = os.path.join(out_path, "valid.csv")
if os.path.exists(tmp_file):
    os.remove(tmp_file)

Working on ./pruebas/no_ResNet1d/
No weights provided, using random initialization
Start training...


100%|██████████| 1/1 [00:09<00:00,  9.33s/it]


epoch,train_f1,train_loss,valid_f1,valid_f1_post,valid_loss

0,0.0010,0.2791,0.0000,0.0000,0.0805



100%|██████████| 1/1 [00:09<00:00,  9.75s/it]


1,0.0000,0.2394,0.0000,0.0000,0.0827



...

3,0.0200,0.0966,0.0000,0.0000,0.0591



100%|██████████| 1/1 [00:09<00:00,  9.08s/it]


4,0.1881,0.0715,0.1517,0.1488,0.0779



100%|██████████| 1/1 [00:08<00:00,  8.79s/it]

5,0.2540,0.0657,0.0930,0.1825,0.0935

